Weighting sentences by their position in the review:

We calculate feature importance off of the test set's features because only the test set's sentences get scored (probssentences). Is this legal?

In [1]:
import pickle
import datetime
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

In [2]:
probssentences = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/pickled_objects/probssentences.p', 'rb'))
test1 = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/pickled_objects/test1.p', 'rb'))

In [3]:
#append doc value to test1
test1['doc'] = [ i for i in range(len(test1))]
print test1.shape
print probssentences.shape

(12500, 4)
(133515, 3)


In [4]:
test1[0:10]

,id,sentiment,review,doc
21130,"""5706_2""",0,"""This movie is like the thousand \""cat and mou...",0
3404,"""11132_3""",0,"""A documentarist, like any filmmaker, must con...",1
14560,"""10770_7""",1,"""With a well thought out cast, this movie was ...",2
6671,"""1058_10""",1,"""this is the best sci-fi that I have seen in m...",3
10982,"""12239_1""",0,"""Ever since I was eight years old I have been ...",4
5526,"""966_7""",1,"""2005 Toronto Film Festival Report It is offic...",5
19301,"""11645_2""",0,"""This is a review of 'Freddy Mercury The Untol...",6
17060,"""5920_9""",1,"""American icon Henry Fonda portrays \""Elegant\...",7
24744,"""9501_2""",0,"""This is the page for \""House of Exorcism\"", b...",8
5253,"""7877_4""",0,"""The makers of this film have created a future...",9


In [5]:
#append the true class value
probssentences["truelabel"] = 0
for i in range(len(probssentences)):
    #get true label value from test1 where index matches doc number
    j = probssentences.iloc[i]['doc']
    probssentences.set_value(i, 'truelabel', test1.iloc[[j]]["sentiment"])

In [6]:
probssentences[0:10]

,0,1,doc,truelabel
0,0.5986,0.4014,0,0
1,0.00792023,0.99208,0,0
2,0.369248,0.630752,0,0
3,0.445318,0.554682,0,0
4,0.999581,0.000419095,0,0
5,0.00449226,0.995508,1,0
6,0.918007,0.0819928,1,0
7,0.55223,0.44777,1,0
8,0.986808,0.0131918,1,0
9,0.721142,0.278858,1,0


In [7]:
#split test set into two
fracTrain = 0.5
nSamples = test1.shape[0]
print nSamples
order = np.random.permutation(nSamples) # come up with a random ordering
print len(order)
splitIndex = int(np.round(nSamples*fracTrain))
print len(order[:splitIndex])
print len(order[splitIndex:])
testA = test1.iloc[order[:splitIndex],:]
testB = test1.iloc[order[splitIndex:],:]

12500
12500
6250
6250


In [54]:
#order 2 test sets in order of docs?

In [8]:
#split probssentences the same way
probssentencesA = probssentences.loc[probssentences['doc'].isin(testA['doc'].unique())]
probssentencesB = probssentences.loc[probssentences['doc'].isin(testB['doc'].unique())]

In [9]:
print len(probssentences)
print len(probssentencesA)
print len(probssentencesB)
print len(testA)
print len(testB)

133515
67287
66228
6250
6250


In [10]:
probssentencesA[0:10]

,0,1,doc,truelabel
0,0.5986,0.4014,0,0
1,0.00792023,0.99208,0,0
2,0.369248,0.630752,0,0
3,0.445318,0.554682,0,0
4,0.999581,0.000419095,0,0
5,0.00449226,0.995508,1,0
6,0.918007,0.0819928,1,0
7,0.55223,0.44777,1,0
8,0.986808,0.0131918,1,0
9,0.721142,0.278858,1,0


In [11]:
probssentencesB[0:10]

,0,1,doc,truelabel
16,0.0593864,0.940614,3,1
17,0.0795348,0.920465,3,1
18,0.00578835,0.994212,3,1
35,3.08829e-07,1,5,1
36,0.139833,0.860167,5,1
37,0.193762,0.806238,5,1
38,0.666227,0.333773,5,1
39,0.0583559,0.941644,5,1
40,0.000877276,0.999123,5,1
41,0.602586,0.397414,5,1


In [12]:
testA[0:10]

,id,sentiment,review,doc
11462,"""252_9""",1,"""The performances were superb, the costumes de...",11991
2223,"""7651_8""",1,"""An interesting TV movie based on true fact, b...",8352
23972,"""2956_8""",1,"""There are very few performers today who can k...",4394
8913,"""11904_10""",1,"""I gave it a 10, since everyone else seemed to...",11368
3891,"""10947_8""",1,"""The Sarah Silverman program is very similar t...",4806
18172,"""10034_1""",0,"""If there is one film which is the worst of th...",2623
19396,"""5576_3""",0,"""To this day when you speak of the Japanese ci...",12171
20284,"""8359_4""",0,"""Despite some moments in heavy rain, an encoun...",11703
13237,"""1465_3""",0,"""Well first off I'd like to add that I myself ...",4973
21651,"""4873_8""",1,"""For those of you who are not aware with the t...",11710


In [13]:
#checking that testB matches probssentencesB
print testB.loc[testB['doc']==2]

Empty DataFrame
Columns: [id, sentiment, review, doc]
Index: []


### NOTE: probssentences and test set are not in same order. Must match on doc number.

In [29]:
print len(testA['doc'].unique())

6250


In [30]:
a = datetime.datetime.now().replace(microsecond=0)

#put into function and make split number the parameter
def getFeaturesAndLabels(numSplits):
    i = 0
    featuresNeg = []
    featuresPos = []
    labels = []
    #for each doc in probssentences:
    for doc in testA['doc'].unique():  
        avgScoresPos = []
        avgScoresNeg = []
        df = probssentencesA.loc[probssentencesA['doc'] == doc]
        if len(df) >= numSplits:
            #split into x groups
            lst = np.array_split(df, numSplits)
            for z in lst:
                #make the average score for each group the feature
                avgScoresNeg.append(z[[0]].mean()[0])
                avgScoresPos.append(z[[1]].mean()[1])
                #convert to a 0 or 1 (?)
            featuresNeg.append(avgScoresNeg)
            featuresPos.append(avgScoresPos)
            labels.append(testA.loc[testA['doc'] == doc]['sentiment'].values[0])
        i = i + 1
    return (featuresNeg, featuresPos, labels)
    
#function call
x = 3
tuple1 = getFeaturesAndLabels(x)

b = datetime.datetime.now().replace(microsecond=0)
print b-a

0:01:24


In [82]:
print len(tuple1)

3


In [31]:
#ignore those reviews with fewer sentences than splits, so instead of 6250 datapoints to build on, have lost about 242:
print len(tuple1[0])
print len(tuple1[1])
print len(tuple1[2])
featuresPos = tuple1[0]
featuresNeg = tuple1[1]
labels = tuple1[2]

6009
6009
6009


In [32]:
#perform feature weighting-
clf = ExtraTreesClassifier()
clf = clf.fit(featuresNeg, labels)
weightsNeg = clf.feature_importances_  
print weightsNeg
clf = ExtraTreesClassifier()
clf = clf.fit(featuresPos, labels)
weightsPos = clf.feature_importances_
print weightsPos

[ 0.37734511  0.2849706   0.33768429]
[ 0.27535623  0.36157908  0.36306469]


### Split test set 2 into x subsets as well

In [49]:
#try to order
testB = testB.sort(columns='doc')
print testB['doc'][0:20]
print probssentencesB[0:25]

6671      3
5526      5
19301     6
21967    10
19991    12
18997    13
10666    14
1898     15
7095     19
4000     20
5440     21
11623    22
9706     23
1694     24
571      25
9950     26
12458    28
13710    29
13269    30
13529    31
Name: doc, dtype: int64
              0           1 doc  truelabel
16    0.0593864    0.940614   3          1
17    0.0795348    0.920465   3          1
18   0.00578835    0.994212   3          1
35  3.08829e-07           1   5          1
36     0.139833    0.860167   5          1
37     0.193762    0.806238   5          1
38     0.666227    0.333773   5          1
39    0.0583559    0.941644   5          1
40  0.000877276    0.999123   5          1
41     0.602586    0.397414   5          1
42     0.680643    0.319357   5          1
43     0.370615    0.629385   5          1
44     0.633409    0.366591   5          1
45     0.155581    0.844419   5          1
46     0.526781    0.473219   5          1
47     0.615371    0.384629   6          0
48   

/home/sarahwie/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [33]:
a = datetime.datetime.now().replace(microsecond=0)

#numSplits variable to be consistent with training set
x = 3

i = 0
featuresTestPos = []
featuresTestNeg = []
labelsTest = []
#for each doc in probssentences:
for doc in testB['doc'].unique():
    avgScoresTestPos = []
    avgScoresTestNeg = []
    df = probssentencesB.loc[probssentencesB['doc'] == doc]
    if len(df) >= x:
        #split into x groups
        lst = np.array_split(df, x)
        for z in lst:
            avgScoresTestNeg.append(z[[0]].mean()[0])
            avgScoresTestPos.append(z[[1]].mean()[1])
        featuresTestNeg.append(avgScoresTestNeg)
        featuresTestPos.append(avgScoresTestPos)
        labelsTest.append(testB.loc[testB['doc'] == doc]['sentiment'].values[0])
    i = i + 1

b = datetime.datetime.now().replace(microsecond=0)
print b-a

0:01:25


In [35]:
print (featuresTestPos*weightsPos).shape
print (featuresTestPos*weightsPos)[0]
print featuresTestPos[0]
print weightsPos

(6014, 3)
[ 0.2590038   0.33282095  0.36096313]
[0.94061356782913208, 0.92046517133712769, 0.99421161413192749]
[ 0.27535623  0.36157908  0.36306469]


In [36]:
print (featuresTestNeg*weightsNeg).shape
print (featuresTestNeg*weightsNeg)[0]
print featuresTestNeg[0]
print weightsNeg

(6014, 3)
[ 0.02240918  0.02266507  0.00195463]
[0.059386439621448517, 0.079534783959388733, 0.0057883495464920998]
[ 0.37734511  0.2849706   0.33768429]


In [37]:
#get probs for each doc in test2:
probsThreePos = featuresTestPos * weightsPos
print probsThreePos.shape
#combine into one prob per review
probPos = probsThreePos.mean(1)
print probsThreePos[0]
print probPos[0]
print probPos.shape

(6014, 3)
[ 0.2590038   0.33282095  0.36096313]
0.317595962392
(6014,)


In [38]:
probsThreeNeg = featuresTestNeg * weightsNeg
print probsThreeNeg.shape
#combine into one prob per review
probNeg = probsThreeNeg.mean(1)
print probsThreeNeg[0]
print probNeg[0]
print probNeg.shape

(6014, 3)
[ 0.02240918  0.02266507  0.00195463]
0.0156762974252
(6014,)


In [ ]:
#labelsTest is in same order as the doc probs

In [39]:
preds = np.ones((probPos.shape[0]))
print preds.shape

(6014,)


In [40]:
preds[np.where(probPos < probNeg)] = 0 # The first column is the negative model

In [41]:
print preds[0:10]
print probPos[0:10]
print probNeg[0:10]

[ 1.  1.  0.  1.  1.  0.  1.  0.  0.  1.]
[ 0.31759596  0.21891827  0.15087839  0.22911343  0.24748008  0.0546729
  0.30589389  0.11112584  0.0437495   0.16956054]
[ 0.0156763   0.11077552  0.17852998  0.09435523  0.0913307   0.27048462
  0.0370252   0.21975946  0.28702394  0.16177221]


In [42]:
print type(preds)
print preds.shape
#convert labels also to numpy array
labelsTest = np.asarray(labelsTest)
print labelsTest.shape

<type 'numpy.ndarray'>
(6014,)
(6014,)


In [43]:
print np.size(np.where(preds == labelsTest))*1./np.size(preds)

0.864316594613


In [44]:
from sklearn.metrics import roc_auc_score
print roc_auc_score(labelsTest, preds)

0.864013308451


Essentially same performance. Why??

In [112]:
print test1['review'].iloc[0]

"This movie is like the thousand \"cat and mouse\" movies that preceded it. (The following may look like a spoiler, but it really just describes a large class of movies) There is the passionate, wise main character, his goofy but well-meaning sidekick with his ill-placed attempts at humorous comments, the initially-hostile but soon softened gorgeous lady who triggers the inevitable \"unlikely\" love story, the loved ones taken hostage, and of course the careless evil adversary with his brutal minions. Everybody has seen tons of these movies already, and \"National Treasure\" is like any one of them, with only a slightly modified wrapping. Every turn of the story was easily predicted (and I can assure you I am not the sharpest tool in the shed). I am quite tired of feeling tricked for money after exiting the theater from a Hollywood movie, and if you have ever felt that way too, heed my warning; stay miles away from this movie."


In [111]:
print test1[0:12]

              id  sentiment  \
21130   "5706_2"          0   
3404   "11132_3"          0   
14560  "10770_7"          1   
6671   "1058_10"          1   
10982  "12239_1"          0   
5526     "966_7"          1   
19301  "11645_2"          0   
17060   "5920_9"          1   
24744   "9501_2"          0   
5253    "7877_4"          0   
21967   "5442_8"          1   
18609   "7027_4"          0   

                                                  review  doc  
21130  "This movie is like the thousand \"cat and mou...    0  
3404   "A documentarist, like any filmmaker, must con...    1  
14560  "With a well thought out cast, this movie was ...    2  
6671   "this is the best sci-fi that I have seen in m...    3  
10982  "Ever since I was eight years old I have been ...    4  
5526   "2005 Toronto Film Festival Report It is offic...    5  
19301  "This is a review of 'Freddy Mercury The Untol...    6  
17060  "American icon Henry Fonda portrays \"Elegant\...    7  
24744  "This is the 

In [57]:
#This review is still scored negatively even though last sentence should have been most heavily weighted.
print testB.loc[testB['doc']==22]
#in position 11
print testB.iloc[11]
print preds[11]
print featuresTestNeg[11]
print weightsNeg
print featuresTestPos[11]
print weightsPos
print probssentencesB.loc[probssentencesB['doc']==22]

             id  sentiment                                             review  \
11623  "8983_8"          1  "Shot into car from through the windscreen, so...   

       doc  
11623   22  
id                                                    "8983_8"
sentiment                                                    1
review       "Shot into car from through the windscreen, so...
doc                                                         22
Name: 11623, dtype: object
0.0
[0.85332351922988892, 0.70063495635986328, 0.47064865659922361]
[ 0.37734511  0.2849706   0.33768429]
[0.1466764829820022, 0.29936500173062086, 0.52935130521655083]
[ 0.27535623  0.36157908  0.36306469]
            0           1 doc  truelabel
269  0.997813  0.00218711  22          1
270  0.708834    0.291166  22          1
271  0.413562    0.586438  22          1
272  0.987708   0.0122923  22          1
273  0.924647   0.0753527  22          1
274   0.01665     0.98335  22          1


In [16]:
print probssentencesA.loc[probssentencesA['doc']==0]

            0            1 doc  truelabel
0      0.5986       0.4014   0          0
1  0.00792023      0.99208   0          0
2    0.369248     0.630752   0          0
3    0.445318     0.554682   0          0
4    0.999581  0.000419095   0          0
